# 使用 torch.autograd 自动微分
- 在训练神经网络时，最常用的算法是反向传播。在该算法中，根据损失函数相对于给定参数的梯度来调整参数（模型权重）。
- 为了计算这些梯度，PyTorch 有一个内置的微分引擎，称为 torch.autograd。它支持任何计算图的梯度自动计算。
- 考虑最简单的一层神经网络，具有输入 x、参数 w 和 b 以及一些损失函数。它可以通过以下方式在 PyTorch 中定义：

In [4]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True) # 随机初始化权重
print(w)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b # 预测值z=wx+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y) # 计算损失
loss

tensor([[ 0.1947,  0.8704, -0.3393],
        [ 1.9837,  0.6916,  0.2246],
        [-0.8458,  0.7288, -0.0619],
        [-2.4849, -1.1634, -0.9581],
        [-1.2200, -0.9672, -1.1967]], requires_grad=True)


tensor(0.3250, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [2]:
# 在这个网络中，w和b是我们需要优化的参数。
# 因此，我们需要能够计算损失函数相对于这些变量的梯度。
# 为此，我们设置这些张量的 require_grad 属性。
# 您可以在创建张量时设置requires_grad的值，或者稍后使用x.requires_grad_(True)方法设置。
# 我们应用于张量来构造计算图的函数实际上是类 Function 的对象。
# 该对象知道如何向前计算函数，以及如何在向后传播步骤中计算其导数。
# 对反向传播函数的引用存储在张量的 grad_fn 属性中。
print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x00000226EA4C38B0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x00000226D276BB80>


## 1、计算梯度

In [3]:
# 反向传播 优化权重
loss.backward()
print(w.grad)
print(b.grad)
# https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html

tensor([[0.3175, 0.1440, 0.0126],
        [0.3175, 0.1440, 0.0126],
        [0.3175, 0.1440, 0.0126],
        [0.3175, 0.1440, 0.0126],
        [0.3175, 0.1440, 0.0126]])
tensor([0.3175, 0.1440, 0.0126])


## 2、禁用反向传播
- 默认情况下，所有 require_grad=True 的张量都会跟踪其计算历史并支持梯度计算。
- 然而，在某些情况下我们不需要这样做，例如，当我们训练了模型并且只想将其应用于某些输入数据时，即我们只想通过网络进行前向计算。
- 我们可以通过用 torch.no_grad() 块包围我们的计算代码来停止跟踪计算：

In [5]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [6]:
# 实现相同结果的另一种方法是在张量上使用 detach() 方法：
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


## 3、雅可比矩阵和雅可比乘积

In [7]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])
